In [5]:
import pandas as pd
from itertools import product

Edit the cell below with the path to your experiment

In [7]:
df = pd.read_pickle('results/CIFAR10N-2024-Jan-05-15-35-50/results.pkl')

In [8]:
df

cp            \
                                                             cvg             
                                                            mean       std   
logits_dir n_cal n_test alpha noise_type    model_name                       
models     10000 5000   0.1   clean_label   logits      0.899672  0.005119   
                              worse_label   logits      0.990256  0.001137   
                              aggre_label   logits      0.940208  0.002836   
                              random_label1 logits      0.972872  0.002322   
                              random_label2 logits      0.977088  0.002353   
                              random_label3 logits      0.973048  0.002246   

                                                                            \
                                                            size             
                                                            mean       std   
logits_dir n_cal n_test alpha noise_type    model_name                       
models     10000 5000   0.1   clean_label   logits      1.507280  0.019043   
                              worse_label   logits      5.472944  0.077880   
                              aggre_label   logits      2.002968  0.027047   
                              random_label1 logits      2.997144  0.052930   
                              random_label2 logits      3.176912  0.066038   
                              random_label3 logits      3.042472  0.079477   

                                                       cp_adjusted            \
                                                               cvg             
                                                              mean       std   
logits_dir n_cal n_test alpha noise_type    model_name                         
models     10000 5000   0.1   clean_label   logits        0.899312  0.004632   
                              worse_label   logits        0.906856  0.008758   
                              aggre_label   logits        0.889128  0.005103   
                              random_label1 logits        0.892448  0.004568   
                              random_label2 logits        0.892656  0.005835   
                              random_label3 logits        0.887856  0.005751   

                                                                            
                                                            size            
                                                            mean       std  
logits_dir n_cal n_test alpha noise_type    model_name                      
models     10000 5000   0.1   clean_label   logits      1.507008  0.018673  
                              worse_label   logits      2.188608  0.092928  
                              aggre_label   logits      1.550488  0.019125  
                              random_label1 logits      1.671800  0.021771  
                              random_label2 logits      1.658184  0.020832  
                              random_label3 logits      1.636184  0.027193

In [32]:
by_noise_type = df.unstack('noise_type').reset_index(drop=True).stack()

In [35]:
by_noise_type.index = by_noise_type.index.droplevel(0)

In [43]:
def combine_mean_std(row):
    return f'{row["mean"]: .3f} ± {row["std"]: .3f}'

table = []
for comb in product(['cp', 'cp_adjusted'], ['cvg', 'size']):
    aggr = by_noise_type.loc[:, comb].apply(combine_mean_std, axis=1)
    aggr.name = comb
    table.append(aggr)

table = pd.concat(table, axis=1)

In [45]:
col_d = dict(zip(table.columns.levels[0], ['CP', 'CRCP']))
col_d.update(zip(table.columns.levels[1], ['Coverage', 'Size']))

row_d = dict(zip(table.index, ['Aggr', 'Clean', 'R1', 'R2', 'R3', 'Worst']))
table = table.rename(columns=col_d).rename(row_d)
table.index.name = 'Noise Type'


In [47]:
print_order = ['Clean', 'Aggr', 'R1', 'R2', 'R3', 'Worst']
table = table.loc[print_order]

In [50]:
latex = table.to_latex(formatters={'Noise Type': lambda s: f'\\texttt{s}'})
print(latex)

\begin{tabular}{lllll}
\toprule
{} & \multicolumn{2}{l}{CP} & \multicolumn{2}{l}{CRCP} \\
{} &         Coverage &             Size &         Coverage &             Size \\
Noise Type &                  &                  &                  &                  \\
\midrule
Clean      &   0.900 ±  0.005 &   1.507 ±  0.019 &   0.909 ±  0.005 &   1.507 ±  0.019 \\
Aggr       &   0.940 ±  0.003 &   2.003 ±  0.027 &   0.899 ±  0.005 &   1.550 ±  0.019 \\
R1         &   0.973 ±  0.002 &   2.997 ±  0.053 &   0.902 ±  0.005 &   1.672 ±  0.022 \\
R2         &   0.977 ±  0.002 &   3.177 ±  0.066 &   0.903 ±  0.006 &   1.658 ±  0.021 \\
R3         &   0.973 ±  0.002 &   3.042 ±  0.079 &   0.898 ±  0.006 &   1.636 ±  0.027 \\
Worst      &   0.990 ±  0.001 &   5.473 ±  0.078 &   0.917 ±  0.009 &   2.189 ±  0.093 \\
\bottomrule
\end{tabular}
